In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.graph_objs as go
import pickle
from urllib.request import urlopen
import json
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

# Часть 1. Предобработка данных

In [130]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [131]:
train.shape

(30471, 292)

In [132]:
def preprocessing_date(data):
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['year'] = data['timestamp'].dt.year
    data['month'] = data['timestamp'].dt.month

    return data

In [133]:
train = preprocessing_date(train)
test = preprocessing_date(test)

In [134]:
target = train['price_doc']
id_test = test['id']

Удалим признаки не содержащие информации

In [135]:
dropped_feat = ['price_doc', 'timestamp', 'id', 'ID_metro', 
                'raion_build_count_with_material_info',
                'raion_build_count_with_builddate_info',
                'ID_railroad_station_walk', "ID_railroad_station_avto",
                "ID_big_road1", "ID_big_road2", "ID_railroad_terminal",
                "ID_bus_terminal"]

In [136]:
for feat in dropped_feat:
    if feat in test.columns:
        test = test.drop(feat, axis=1)
    train = train.drop(feat, axis=1)

Проверим на наличие пропущеных значений

In [137]:
train.isna().sum()[train.isna().sum() > 0]

life_sq                         6383
floor                            167
max_floor                       9572
material                        9572
build_year                     13605
num_room                        9572
kitch_sq                        9572
state                          13559
preschool_quota                 6688
school_quota                    6685
hospital_beds_raion            14441
build_count_block               4991
build_count_wood                4991
build_count_frame               4991
build_count_brick               4991
build_count_monolith            4991
build_count_panel               4991
build_count_foam                4991
build_count_slag                4991
build_count_mix                 4991
build_count_before_1920         4991
build_count_1921-1945           4991
build_count_1946-1970           4991
build_count_1971-1995           4991
build_count_after_1995          4991
metro_min_walk                    25
metro_km_walk                     25
r

Разделим данные на категориальные и вещественные

In [138]:
cat_feat = ['material', 'state', 'product_type', 'sub_area', 'culture_objects_top_25', 
 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 
 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 
 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 
 'big_road1_1line', 'railroad_1line', 'ecology']

In [139]:
num_feat = [x for x in train.columns if x not in cat_feat]

Заполним пропуски данных

In [140]:
imp_most_frequent = SimpleImputer(strategy='most_frequent')
imp_median = SimpleImputer(strategy='median')

# заполнение категориальных признаков
cat_train = pd.DataFrame(imp_most_frequent.fit_transform(train[cat_feat]), 
                         columns=cat_feat)

cat_test = pd.DataFrame(imp_most_frequent.transform(test[cat_feat]),
                        columns=cat_feat)

cat_train = cat_train.astype(train[cat_feat].dtypes)
cat_test = cat_test.astype(test[cat_feat].dtypes)

# заполненеие вещественных признаков
num_train = pd.DataFrame(imp_median.fit_transform(train[num_feat]),
                 columns=num_feat)

num_test = pd.DataFrame(imp_median.transform(test[num_feat]),
                 columns=num_feat)

num_train = num_train.astype(train[num_feat].dtypes)
num_test = num_test.astype(test[num_feat].dtypes)

# объединим заполненые данные
train = pd.concat([num_train, cat_train], axis=1)
test = pd.concat([num_test, cat_test], axis=1)

Откорректируем данные

In [141]:
def correct_values(data):
  # приравниваем жилую площадь к общей, там где она превышает общую
  bad_index = data[data['full_sq'] < data['life_sq']].index
  data.loc[bad_index, ['life_sq']] = data.loc[bad_index, ['full_sq']]
  
  # убираем слишком большие и маленькие годы постройки
  data.loc[data['build_year']<1500, 'build_year'] = np.NaN
  data.loc[data['build_year']>2020, 'build_year'] = np.NaN
  
  # убираем нулевые значения количества комнат
  data.loc[data['num_room']==0, 'num_room'] = np.NaN
  
  # приравниваем этажность здания к этажу объекта там где она меньше
  bad_index = data[data['max_floor'] < data['floor']].index
  data.loc[bad_index, ['max_floor']] = data.loc[bad_index, ['floor']]
  
  # исправляем опечатку в данных
  data['state'] = data['state'].replace(33,3)
  
  # убираем площадь кухни где она больше общей площади
  bad_index = data[data['full_sq'] < data['kitch_sq']].index
  data.loc[bad_index, ['kitch_sq']] = np.NaN
  
  # корректировка типа данных
  data['material'] = data['material'].astype('int64')
  data['state'] = data['state'].astype('int64')

  return data

In [142]:
train = correct_values(train)
test = correct_values(test)

In [143]:
# выгрузим обработанные данные
train.to_csv('processed_data.csv', index=False)

Для выполнения задания школы DS разделим тренировочные данные на тренировочные и тестовые, т.к. они имеют целевую переменную и далее будем работать с ними.

In [144]:
def split_data(data, target, train_share):
  share = round(len(data)*train_share)
  X_train, y_train = data.iloc[:share,:], target[:share]
  X_test, y_test = data.iloc[share:,:], target[share:]

  return X_train, X_test, y_train, y_test

In [145]:
X_train, X_test, y_train, y_test = split_data(train, target, train_share=0.8)

# Часть 2. Расчет модели

Будем использовать алгоритм градиентного бустинга из библиотеки CatBoost. В данном алгоритме кодировка категориальных признаков проводится внутри самого алгоритма, поэтому ненужно предварительно подготавливать данные. Кроме этого алгоритм основа на Decision Trees для которых нет необходимости проводить масштабирование вещественных признаков.

In [146]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_log_error

In [147]:
train_dataset = Pool(X_train, y_train, cat_features=cat_feat)                                                      
test_dataset = Pool(X_test, y_test, cat_features=cat_feat)

In [148]:
# выбор оптимального количества деревьев
model = CatBoostRegressor(iterations=5000)
model.fit(train_dataset, 
          use_best_model=True,
          eval_set=test_dataset, silent=True)
best_iter = model.tree_count_
best_iter

4755

In [149]:
mean_squared_log_error(y_test, model.predict(X_test))**0.5

0.40821152729773036

In [150]:
# обучаем модель со всеми фичами
model = CatBoostRegressor(iterations=best_iter)
model.fit(train_dataset, silent=True)

In [151]:
mean_squared_log_error(y_test, model.predict(X_test))**0.5

0.4075278724367229

Рассмотрим важность признаков и посмотрим как будет изменяться метрика RMSLE при уменьшении количества признаков.

In [152]:
feature = model.feature_names_
importance = model.feature_importances_
rank = np.arange(1, len(feature) + 1)
name = ['feature', 'importance']
df = pd.DataFrame(zip(feature, importance), 
                  columns=name).sort_values('importance', ascending=False)
df['rank'] = rank

In [76]:
best_count_feat = len(df)
best_RMSLE = 1
score = {}
for i in range(len(df), 0, -20):
    columns = df['feature'][0:i]
    temp_cat_feat = [x for x in cat_feat if x in columns.to_list()]
    train_dataset = Pool(X_train[columns], y_train, cat_features=temp_cat_feat)  
    model = CatBoostRegressor(iterations=best_iter)
    model.fit(train_dataset, silent=True)
    RMSLE = mean_squared_log_error(y_test, model.predict(X_test[columns]))**0.5
    score[len(columns)] = round(RMSLE,4)
    if RMSLE < best_RMSLE:
        best_RMSLE = RMSLE
        best_count_feat = i

In [26]:
import json
with open('score.json', 'w') as outfile:
    json.dump(score, outfile)

In [90]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(score.keys()),
    y=list(score.values()),
    mode="lines+markers+text",
    name="Lines, Markers and Text",
    text=list(score.values()),
    textposition="top center"
))
fig.update_xaxes(title_text='Count of features')
fig.update_yaxes(title_text='RMSLE')
fig.show()

Вывод: можем сократить количество признаков до ** (отобрав наибоолее важные) без снижения качества модели.

In [173]:
best_count_feat = 122+1

In [174]:
pd.set_option('max_rows', None)
print(f'{best_count_feat} most important features')
df.head(best_count_feat)
#df[df['feature'] == 'material']

123 most important features


,feature,importance,rank
0,full_sq,37.373690,1
263,year,1.996224,2
4,build_year,1.895479,3
266,state,1.697949,4
1,life_sq,1.565857,5
2,floor,1.350315,6
268,sub_area,1.341102,7
254,cafe_count_5000_price_2500,1.265242,8
231,cafe_count_3000_price_2500,1.138517,9
256,cafe_count_5000_price_high,1.124648,10


In [175]:
# обучим модель на ** признаках
columns = df['feature'][0:best_count_feat]
temp_cat_feat = [x for x in cat_feat if x in columns.to_list()]
train_dataset = Pool(X_train[columns], y_train, cat_features=temp_cat_feat)  
best_model = CatBoostRegressor(iterations=best_iter)
best_model.fit(train_dataset, silent=True)
RMSLE = mean_squared_log_error(y_test, best_model.predict(X_test[columns]))**0.5
print(RMSLE)

0.40755954496995905


Сохранение модели для выполнения задания школы DS

In [176]:
pkl_filename = "pickle_model.pkl" 
with open(pkl_filename, 'wb') as file:
    pickle.dump(best_model, file)

Для оценки модели на kagle, переобучим модель на неразделенной выборке train, и предскажем результаты по выборке test.

In [177]:
train_dataset = Pool(train[columns], target, cat_features=temp_cat_feat) 
best_model.fit(train_dataset)
RMSLE = mean_squared_log_error(target, best_model.predict(train[columns]))**0.5
print(RMSLE)

Learning rate set to 0.019775
0:	learn: 4732592.9581669	total: 50.3ms	remaining: 3m 59s
1:	learn: 4684965.9326350	total: 101ms	remaining: 4m
2:	learn: 4639133.8076010	total: 165ms	remaining: 4m 21s
3:	learn: 4594334.3589050	total: 228ms	remaining: 4m 31s
4:	learn: 4549746.6663299	total: 278ms	remaining: 4m 23s
5:	learn: 4507063.8649714	total: 332ms	remaining: 4m 22s
6:	learn: 4465021.6871645	total: 382ms	remaining: 4m 19s
7:	learn: 4422904.3187190	total: 430ms	remaining: 4m 14s
8:	learn: 4382339.2428655	total: 483ms	remaining: 4m 14s
9:	learn: 4342812.8820257	total: 534ms	remaining: 4m 13s
10:	learn: 4306636.3151813	total: 587ms	remaining: 4m 13s
11:	learn: 4270776.8153757	total: 640ms	remaining: 4m 12s
12:	learn: 4235068.7386874	total: 705ms	remaining: 4m 17s
13:	learn: 4201106.6562147	total: 756ms	remaining: 4m 16s
14:	learn: 4167134.9144163	total: 811ms	remaining: 4m 16s
15:	learn: 4132697.8251166	total: 868ms	remaining: 4m 17s
16:	learn: 4101177.1624477	total: 922ms	remaining: 4m 1

144:	learn: 2763521.2205945	total: 7.64s	remaining: 4m 3s
145:	learn: 2760627.0096185	total: 7.7s	remaining: 4m 3s
146:	learn: 2757502.0237422	total: 7.74s	remaining: 4m 2s
147:	learn: 2754981.3513102	total: 7.79s	remaining: 4m 2s
148:	learn: 2752312.5923009	total: 7.84s	remaining: 4m 2s
149:	learn: 2749800.5540673	total: 7.9s	remaining: 4m 2s
150:	learn: 2746890.8452047	total: 7.96s	remaining: 4m 2s
151:	learn: 2743400.3850852	total: 8.01s	remaining: 4m 2s
152:	learn: 2740803.4649844	total: 8.06s	remaining: 4m 2s
153:	learn: 2738335.4475212	total: 8.12s	remaining: 4m 2s
154:	learn: 2735797.5963647	total: 8.18s	remaining: 4m 2s
155:	learn: 2732693.2176283	total: 8.24s	remaining: 4m 2s
156:	learn: 2730254.5023583	total: 8.29s	remaining: 4m 2s
157:	learn: 2728233.2897130	total: 8.34s	remaining: 4m 2s
158:	learn: 2726106.8849548	total: 8.38s	remaining: 4m 2s
159:	learn: 2724014.3267546	total: 8.43s	remaining: 4m 2s
160:	learn: 2722229.2675055	total: 8.49s	remaining: 4m 2s
161:	learn: 2719

289:	learn: 2542575.4644158	total: 15.1s	remaining: 3m 52s
290:	learn: 2541868.3482113	total: 15.1s	remaining: 3m 51s
291:	learn: 2540810.1683942	total: 15.2s	remaining: 3m 51s
292:	learn: 2539956.3116058	total: 15.2s	remaining: 3m 51s
293:	learn: 2539236.4315912	total: 15.3s	remaining: 3m 51s
294:	learn: 2538594.8957475	total: 15.3s	remaining: 3m 51s
295:	learn: 2538065.9491029	total: 15.4s	remaining: 3m 51s
296:	learn: 2537302.9128068	total: 15.4s	remaining: 3m 51s
297:	learn: 2536044.5718816	total: 15.4s	remaining: 3m 51s
298:	learn: 2535610.8362097	total: 15.5s	remaining: 3m 50s
299:	learn: 2534830.4010428	total: 15.5s	remaining: 3m 50s
300:	learn: 2533833.2834596	total: 15.6s	remaining: 3m 50s
301:	learn: 2532502.3025688	total: 15.7s	remaining: 3m 50s
302:	learn: 2531915.0120396	total: 15.7s	remaining: 3m 50s
303:	learn: 2530934.3384941	total: 15.8s	remaining: 3m 50s
304:	learn: 2529793.9309150	total: 15.8s	remaining: 3m 50s
305:	learn: 2529332.1235295	total: 15.8s	remaining: 3m 5

430:	learn: 2442611.0418299	total: 22.1s	remaining: 3m 41s
431:	learn: 2442086.3603058	total: 22.2s	remaining: 3m 41s
432:	learn: 2441902.0763460	total: 22.2s	remaining: 3m 41s
433:	learn: 2441700.2204142	total: 22.3s	remaining: 3m 41s
434:	learn: 2441278.4297650	total: 22.3s	remaining: 3m 41s
435:	learn: 2440821.6574904	total: 22.4s	remaining: 3m 41s
436:	learn: 2440449.8549713	total: 22.4s	remaining: 3m 41s
437:	learn: 2440016.9221152	total: 22.5s	remaining: 3m 41s
438:	learn: 2439574.6184070	total: 22.5s	remaining: 3m 41s
439:	learn: 2438992.5278425	total: 22.6s	remaining: 3m 41s
440:	learn: 2438535.8286308	total: 22.6s	remaining: 3m 41s
441:	learn: 2437755.9269188	total: 22.7s	remaining: 3m 41s
442:	learn: 2437294.1267180	total: 22.7s	remaining: 3m 41s
443:	learn: 2437093.2508578	total: 22.8s	remaining: 3m 41s
444:	learn: 2436732.0206446	total: 22.8s	remaining: 3m 41s
445:	learn: 2436379.4092964	total: 22.9s	remaining: 3m 41s
446:	learn: 2435885.0631577	total: 22.9s	remaining: 3m 4

570:	learn: 2374401.5886609	total: 29.2s	remaining: 3m 34s
571:	learn: 2374028.7417065	total: 29.3s	remaining: 3m 34s
572:	learn: 2373861.0795431	total: 29.3s	remaining: 3m 34s
573:	learn: 2373792.2331498	total: 29.4s	remaining: 3m 34s
574:	learn: 2373405.4493944	total: 29.4s	remaining: 3m 34s
575:	learn: 2373031.8872876	total: 29.5s	remaining: 3m 33s
576:	learn: 2372905.7777784	total: 29.5s	remaining: 3m 33s
577:	learn: 2372141.1222676	total: 29.6s	remaining: 3m 33s
578:	learn: 2371914.8185540	total: 29.6s	remaining: 3m 33s
579:	learn: 2371857.9700394	total: 29.7s	remaining: 3m 33s
580:	learn: 2371451.5662713	total: 29.7s	remaining: 3m 33s
581:	learn: 2371382.1850915	total: 29.8s	remaining: 3m 33s
582:	learn: 2370819.7502972	total: 29.8s	remaining: 3m 33s
583:	learn: 2370370.4619202	total: 29.9s	remaining: 3m 33s
584:	learn: 2369636.0423046	total: 29.9s	remaining: 3m 33s
585:	learn: 2369598.2400895	total: 30s	remaining: 3m 33s
586:	learn: 2368750.3633975	total: 30s	remaining: 3m 33s
5

...

4171:	learn: 1700782.0361806	total: 4m 19s	remaining: 36.3s
4172:	learn: 1700731.8058868	total: 4m 20s	remaining: 36.3s
4173:	learn: 1700623.4729822	total: 4m 20s	remaining: 36.2s
4174:	learn: 1700582.3217094	total: 4m 20s	remaining: 36.1s
4175:	learn: 1700530.8747630	total: 4m 20s	remaining: 36.1s
4176:	learn: 1700436.5681286	total: 4m 20s	remaining: 36s
4177:	learn: 1700384.6979061	total: 4m 20s	remaining: 36s
4178:	learn: 1700343.0921802	total: 4m 20s	remaining: 35.9s
4179:	learn: 1700256.4938833	total: 4m 20s	remaining: 35.8s
4180:	learn: 1700176.4406697	total: 4m 20s	remaining: 35.8s
4181:	learn: 1700068.1565310	total: 4m 20s	remaining: 35.7s
4182:	learn: 1700026.9310108	total: 4m 20s	remaining: 35.6s
4183:	learn: 1699877.4276565	total: 4m 20s	remaining: 35.6s
4184:	learn: 1699832.5759167	total: 4m 20s	remaining: 35.5s
4185:	learn: 1699755.4629095	total: 4m 20s	remaining: 35.5s
4186:	learn: 1699696.0696046	total: 4m 20s	remaining: 35.4s
4187:	learn: 1699643.3244094	total: 4m 20s	r

4311:	learn: 1688631.3114835	total: 4m 29s	remaining: 27.7s
4312:	learn: 1688553.8552040	total: 4m 29s	remaining: 27.6s
4313:	learn: 1688486.3689019	total: 4m 29s	remaining: 27.6s
4314:	learn: 1688402.1430886	total: 4m 29s	remaining: 27.5s
4315:	learn: 1688305.2701170	total: 4m 29s	remaining: 27.4s
4316:	learn: 1688157.4711583	total: 4m 29s	remaining: 27.4s
4317:	learn: 1688015.8146161	total: 4m 29s	remaining: 27.3s
4318:	learn: 1687934.3934164	total: 4m 29s	remaining: 27.2s
4319:	learn: 1687836.6449888	total: 4m 29s	remaining: 27.2s
4320:	learn: 1687738.6009014	total: 4m 30s	remaining: 27.1s
4321:	learn: 1687651.3283153	total: 4m 30s	remaining: 27.1s
4322:	learn: 1687489.7940672	total: 4m 30s	remaining: 27s
4323:	learn: 1687440.7366668	total: 4m 30s	remaining: 26.9s
4324:	learn: 1687312.6085588	total: 4m 30s	remaining: 26.9s
4325:	learn: 1687168.2124323	total: 4m 30s	remaining: 26.8s
4326:	learn: 1687147.0585370	total: 4m 30s	remaining: 26.8s
4327:	learn: 1687084.6803313	total: 4m 30s

4449:	learn: 1676531.5225772	total: 4m 40s	remaining: 19.2s
4450:	learn: 1676511.2003392	total: 4m 40s	remaining: 19.2s
4451:	learn: 1676417.7450670	total: 4m 40s	remaining: 19.1s
4452:	learn: 1676288.1041172	total: 4m 40s	remaining: 19s
4453:	learn: 1676211.4732669	total: 4m 40s	remaining: 19s
4454:	learn: 1676127.6052989	total: 4m 41s	remaining: 18.9s
4455:	learn: 1676065.5000082	total: 4m 41s	remaining: 18.9s
4456:	learn: 1676027.1192487	total: 4m 41s	remaining: 18.8s
4457:	learn: 1675911.3087769	total: 4m 41s	remaining: 18.7s
4458:	learn: 1675857.4646645	total: 4m 41s	remaining: 18.7s
4459:	learn: 1675811.2817965	total: 4m 41s	remaining: 18.6s
4460:	learn: 1675764.2704120	total: 4m 41s	remaining: 18.5s
4461:	learn: 1675716.5802832	total: 4m 41s	remaining: 18.5s
4462:	learn: 1675590.6671223	total: 4m 41s	remaining: 18.4s
4463:	learn: 1675495.3036400	total: 4m 41s	remaining: 18.4s
4464:	learn: 1675408.1982222	total: 4m 41s	remaining: 18.3s
4465:	learn: 1675294.9035312	total: 4m 41s	r

4589:	learn: 1666024.7969169	total: 4m 49s	remaining: 10.4s
4590:	learn: 1666002.6447969	total: 4m 49s	remaining: 10.3s
4591:	learn: 1665854.5127175	total: 4m 49s	remaining: 10.3s
4592:	learn: 1665783.6916891	total: 4m 49s	remaining: 10.2s
4593:	learn: 1665708.9424674	total: 4m 49s	remaining: 10.1s
4594:	learn: 1665616.2191992	total: 4m 49s	remaining: 10.1s
4595:	learn: 1665548.1238182	total: 4m 49s	remaining: 10s
4596:	learn: 1665542.1695258	total: 4m 50s	remaining: 9.97s
4597:	learn: 1665463.5560586	total: 4m 50s	remaining: 9.91s
4598:	learn: 1665349.0907246	total: 4m 50s	remaining: 9.85s
4599:	learn: 1665295.4536829	total: 4m 50s	remaining: 9.78s
4600:	learn: 1665293.7903904	total: 4m 50s	remaining: 9.72s
4601:	learn: 1665252.6420375	total: 4m 50s	remaining: 9.66s
4602:	learn: 1665162.2996246	total: 4m 50s	remaining: 9.59s
4603:	learn: 1665133.3518447	total: 4m 50s	remaining: 9.53s
4604:	learn: 1664985.8241559	total: 4m 50s	remaining: 9.47s
4605:	learn: 1664945.1194377	total: 4m 50s

4726:	learn: 1656209.4146569	total: 4m 58s	remaining: 1.77s
4727:	learn: 1656207.7162498	total: 4m 58s	remaining: 1.71s
4728:	learn: 1656140.8007676	total: 4m 58s	remaining: 1.64s
4729:	learn: 1656028.2126235	total: 4m 58s	remaining: 1.58s
4730:	learn: 1656021.4499107	total: 4m 58s	remaining: 1.52s
4731:	learn: 1655932.2861980	total: 4m 58s	remaining: 1.45s
4732:	learn: 1655785.2839468	total: 4m 59s	remaining: 1.39s
4733:	learn: 1655774.8768069	total: 4m 59s	remaining: 1.33s
4734:	learn: 1655770.2263058	total: 4m 59s	remaining: 1.26s
4735:	learn: 1655754.6631619	total: 4m 59s	remaining: 1.2s
4736:	learn: 1655695.6167181	total: 4m 59s	remaining: 1.14s
4737:	learn: 1655676.4004124	total: 4m 59s	remaining: 1.07s
4738:	learn: 1655569.8691447	total: 4m 59s	remaining: 1.01s
4739:	learn: 1655417.7572031	total: 4m 59s	remaining: 948ms
4740:	learn: 1655364.7855829	total: 4m 59s	remaining: 885ms
4741:	learn: 1655189.2829476	total: 4m 59s	remaining: 822ms
4742:	learn: 1655185.3637676	total: 4m 59

In [49]:
pred_target = best_model.predict(test[columns])

In [51]:
# выгрузим результаты
predicted_result = pd.DataFrame({'id': id_test, 'price_doc': pred_target})
predicted_result.to_csv('kaggle_submission.csv', index=False)

# Часть 3. Создание дэшборда

In [178]:
# загружаем ранее выгруженные обработанные данные
df_data = pd.read_csv('processed_data.csv')

In [179]:
# загружаем модель
pkl_filename = "pickle_model.pkl" 
with open(pkl_filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [180]:
# формируем датасет из наиболее часто встречаемых значений
most_freq_df = df_data.mode()[loaded_model.feature_names_]

In [94]:
'''from scipy.stats import skew, mode
most_freq_df = most_freq_df.groupby('sub_area').agg(mode)
col = most_freq_df.columns
for column in most_freq_df.columns:
    most_freq_df[column] = most_freq_df[column].str[0].str[0]
most_freq_df = most_freq_df.reset_index()
most_freq_df['year'] = 2022
most_freq_df = most_freq_df.reindex(columns=loaded_model.feature_names_)'''

"from scipy.stats import skew, mode\nmost_freq_df = most_freq_df.groupby('sub_area').agg(mode)\ncol = most_freq_df.columns\nfor column in most_freq_df.columns:\n    most_freq_df[column] = most_freq_df[column].str[0].str[0]\nmost_freq_df = most_freq_df.reset_index()\nmost_freq_df['year'] = 2022\nmost_freq_df = most_freq_df.reindex(columns=loaded_model.feature_names_)"

In [182]:
# загружаем геоданные Москвы
with urlopen('http://gis-lab.info/data/mos-adm/mo.geojson') as response:
    mosсow_district_geo = json.load(response)

In [183]:
# русско-англиский словарь названий районов
with open('ru_eng_district_names.pkl', 'rb') as f:
    ru_eng_district_names = pickle.load(f)

In [184]:
df_price = pd.DataFrame(zip(ru_eng_district_names.keys(), ru_eng_district_names.values()),
                        columns=['ru_name', 'eng_name'])

In [185]:
color = {'text': '#b3bdc7',
         'background_left': '#1f2630',
         'background_right': '#a7aeb5'}

In [11]:
help(dcc.RadioItems)

Help on class RadioItems in module dash.dcc.RadioItems:

class RadioItems(dash.development.base_component.Component)
 |  RadioItems(options=undefined, value=undefined, inline=undefined, style=undefined, className=undefined, inputStyle=undefined, inputClassName=undefined, labelStyle=undefined, labelClassName=undefined, id=undefined, loading_state=undefined, persistence=undefined, persisted_props=undefined, persistence_type=undefined, **kwargs)
 |  
 |  A RadioItems component.
 |  RadioItems is a component that encapsulates several radio item inputs.
 |  The values and labels of the RadioItems is specified in the `options`
 |  property and the seleced item is specified with the `value` property.
 |  Each radio item is rendered as an input with a surrounding label.
 |  
 |  Keyword arguments:
 |  
 |  - options (list of dicts; optional):
 |      An array of options, or inline dictionary of options.
 |  
 |      `options` is a list of string | number | booleans | dict | list of
 |      dic

In [187]:
# устанавливаем параметры для элементов дэшборда
material_dict = {1: 'монолитный', 2: 'кирпично-монолитный', 3: 'кирпичный', 
                 4: 'панельный', 5: 'блочный', 6: 'деревянный'}

state_dict = {2: 'без ремонта', 3: 'евроремонт', 4: 'дизайнерский ремонт',
              1: 'косметический ремонт'}

reverse_material_dict = dict(zip(material_dict.values(), material_dict.keys()))
reverse_state_dict = dict(zip(state_dict.values(), state_dict.keys()))
                             
min_full_sq = 1
max_full_sq = 200
min_kitch_sq = 1

min_floor = 1
max_floor = df_data['floor'].max()
max_max_floor = df_data['max_floor'].max()

max_rooms = df_data['num_room'].max()

state_apartment = [state_dict[x] for x in sorted(df_data['state'].unique())]
material = [material_dict[x] for x in sorted(df_data['material'].unique())]


app = JupyterDash(__name__)

app.layout = html.Div(children=[


        html.Div(children=[

            #House prices in Moscow
            html.H1('Цены на недвижимость в Москве',
                    style={'textAlign': 'left', 
                           'font-family': 'Helvetica',
                           'font-size': 40,
                           'color': color['text']}),

            html.Label('Укажите параметры расчета',
                       style={'font-family': 'Helvetica',
                              'font-size': 20,
                              'color': color['text']}),

            html.Hr(),
            html.Label('Общая площадь [м2]', style={'font-family': 'Helvetica', 'color': color['text']}),
            dcc.Slider(
                min=min_full_sq,
                max=max_full_sq,
                step=1,
                marks=None,
                value=20,
                id='full_sq',
                tooltip={"placement": "bottom", "always_visible": True},
            ),
            
            html.Label('Жилая площадь [м2]', style={'font-family': 'Helvetica', 'color': color['text']}),
            dcc.Slider(
                min=1,
                step=1,
                marks=None,
                value=10,
                id='life_sq',
                tooltip={"placement": "bottom", "always_visible": True},
            ),
            
            html.Label('Площадь кухни [м2]', style={'font-family': 'Helvetica', 'color': color['text']}),
            dcc.Slider(
                min=1,
                step=1,
                marks=None,
                value=1,
                id='kitch_sq',
                tooltip={"placement": "bottom", "always_visible": True},
            ),
              
            html.Label('Количество комнат', style={'font-family': 'Helvetica', 'color': color['text']}),
            dcc.Dropdown(np.arange(1, max_rooms + 1), value=1, id='num_room'),


            html.Br(),
            html.Div(children=[
                
                html.Div(children=[
                    html.Label('Этаж', style={'font-family': 'Helvetica', 'color': color['text']}),
                    dcc.Dropdown(np.arange(min_floor, max_floor+1), value=1, id='floor')], style={'flex': 1}),

                html.Div(children=[
                    html.Label('Этажность здания', style={'font-family': 'Helvetica', 'color': color['text']}),
                    dcc.Dropdown(id='max_floor')], style={'flex': 1})
                
            ], style={'display': 'flex', 'flex-direction': 'row', 'align-self': 'flex-start', 'margin': 0}),

            html.Br(),
            html.Div(children=[
                
                html.Div(children=[
                    html.Label('Материал стен', style={'font-family': 'Helvetica', 'color': color['text']}),
                    dcc.RadioItems(material,
                                   value=material[0],
                                   id='material',
                                   labelStyle={'display': 'block'},
                                   style={'font-family': 'Helvetica', 'color': color['text']})], style={'flex': 1}),
                html.Div(children=[
                    html.Label('Состояние', style={'font-family': 'Helvetica', 'color': color['text']}),
                    dcc.RadioItems(state_apartment,
                                   value=state_apartment[0],
                                   id='state',
                                   labelStyle={'display': 'block'},
                                   style={'font-family': 'Helvetica', 'color': color['text']})], style={'flex': 1}),
            
            ], style={'display': 'flex', 'flex-direction': 'row', 'align-self': 'flex-start', 'margin': 0}),

            html.Br(),
            html.Label('Расстояние до Садового кольца [км]', style={'font-family': 'Helvetica', 'color': color['text']}),
            dcc.Input(value=1, type='number', id='sadovoe_km'),
            
            html.Br(),
            html.Label('Растояние до метро [км]', style={'font-family': 'Helvetica', 'color': color['text']}),
            dcc.Input(value=1, type='number', id='metro_km_avto'),
            

            
        ], style={'padding': 20, 'flex': 1, 'backgroundColor': color['background_left']}),

        html.Div(children=[
            html.A(),
            dcc.Graph(id='graph')
        ], style={'padding': 20, 'flex': 1, 'backgroundColor': color['background_right']})

], style={'display': 'flex', 'flex-direction': 'row', 'align-self': 'flex-start', 'margin': 0})


@app.callback(
    Output('graph', 'figure'),
    Input('full_sq', 'value'),
    Input('life_sq', 'value'),
    Input('kitch_sq', 'value'),
    Input('num_room', 'value'),
    Input('floor', 'value'),
    Input('max_floor', 'value'),
    Input('material', 'value'),
    Input('state', 'value'),
    Input('metro_km_avto', 'value'),
    Input('sadovoe_km', 'value')
)
def inp_func(full_sq, life_sq, kitch_sq, num_room, floor, max_floor, material, state, metro_km_avto, sadovoe_km):
    # создание заданного объекта
    most_freq_df['full_sq'] = full_sq
    most_freq_df['life_sq'] = life_sq
    most_freq_df['kitch_sq'] = kitch_sq
    most_freq_df['num_room'] = num_room
    most_freq_df['floor'] = floor
    most_freq_df['max_floor'] = max_floor
    most_freq_df['material'] = reverse_material_dict[material]
    most_freq_df['state'] = reverse_state_dict[state]
    most_freq_df['metro_km_avto'] = metro_km_avto
    most_freq_df['sadovoe_km'] = sadovoe_km

    temp_list = []
    for name in df_price['eng_name']:
        most_freq_df['sub_area'] = name
        # temp_most_freq_df = most_freq_df[most_freq_df['sub_area']==name]
        temp_list.append(float(loaded_model.predict(most_freq_df)))
    df_price['price'] = temp_list
    fig = px.choropleth_mapbox(df_price, geojson=mosсow_district_geo, locations="ru_name", color='price',
                               mapbox_style="carto-positron",
                               zoom=8.2, center={"lat": 55.60, "lon": 37.20},
                               featureidkey="properties.NAME",
                               labels={'ru_name': 'district'}
                               )
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0},
                      height=700,
                      width=900)

    return fig

# настройка жилой площади кухни
@app.callback(
    Output('life_sq', 'max'),
    Input('full_sq', 'value')
)
def life_square(full_sq):
    return full_sq

# настройка площади кухни
@app.callback(
    Output('kitch_sq', 'max'),
    Input('full_sq', 'value'),
    Input('life_sq', 'value')
)
def kitch_square(full_sq, life_sq):
    return full_sq - life_sq

# настройка 'этажности'
@app.callback(
    Output('max_floor', 'options'),
    Output('max_floor', 'value'),
    Input('floor', 'value'),
)
def floor(floor):
    return np.arange(floor, 100), floor


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
